In [1]:
import open3d as o3d
import numpy as np

LASER_PATH = "pointclouds/accumulated_laserscan_scene3.ply"
MAP_PATH = "pointclouds/rtabmap_pointcloud_scene3.ply"

laser_pcd = o3d.io.read_point_cloud(LASER_PATH)
map_pcd = o3d.io.read_point_cloud(MAP_PATH)

laser_pcd.paint_uniform_color([1, 0, 0])
map_pcd.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([laser_pcd, map_pcd])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
z = np.array(laser_pcd.points)[0, 2]
slack = 0.1

map_points = np.array(map_pcd.points)
map_points = map_points[map_points[:, 2] >= z - slack]
map_points = map_points[map_points[:, 2] <= z + slack]
map_points[:, 2] = z

map_pcd.points = o3d.utility.Vector3dVector(map_points)
map_pcd.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([laser_pcd, map_pcd])

In [6]:
from scipy.spatial import KDTree
import sensor_msgs.point_cloud2 as pc2

def compute_distances(source_points, target_points):
    tree = KDTree(target_points)
    distances, _ = tree.query(source_points)
    return distances[distances < 0.5]

# Assuming you have the PointCloud2 messages for the laser scan and camera
# laser_scan_cloud and camera_cloud are the PointCloud2 messages

# Convert your ROS PointCloud2 messages to numpy arrays
laser_scan_points = np.asarray(laser_pcd.points)
camera_points = np.asarray(map_pcd.points)

# Compute nearest neighbor distances from camera points to laser scan points
distances = compute_distances(camera_points, laser_scan_points)

# Compute statistics
mean_distance = np.mean(distances)
median_distance = np.median(distances)
std_distance = np.std(distances)

print(f"Mean Distance: {mean_distance}")
print(f"Median Distance: {median_distance}")
print(f"Standard Deviation: {std_distance}")


Mean Distance: 0.09336103625483097
Median Distance: 0.058125739479895674
Standard Deviation: 0.10059548882487157
